<a href="https://colab.research.google.com/github/w95wayne10/personal_financial_management/blob/master/DH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0:  
0:  
[ index:(0, 0), 
$h^2$*1e5 = 10.96, 
$\mu$*1e5 = -5.48, 
$\eta$ = 3, 
$p_u$ = 0.3682, 
$p_m$ = 0.2591, 
$p_d$ = 0.3727, 
price = 0.7225, 
$\Delta$ = 0.50397 ,  
index:(0, 0), 
$h^2$*1e5 = 10.96, 
$\mu$*1e5 = -5.48, 
$\eta$ = 3, 
$p_u$ = 0.3682, 
$p_m$ = 0.2591, 
$p_d$ = 0.3727, 
price=0.7225, 
$\Delta$ = 0.50397 ]  

1:  
-3:  
[ index:(1, -3), 
$h^2$*1e5 = 11.1079, 
$\mu$*1e5 = -5.554, 
$\eta$ = 3, 
$p_u$ = 0.3732, 
$p_m$ = 0.2491, 
$p_d$ = 0.3777, 
price=0.1726, 
$\Delta$ = 0.19247 ,  
index:(1, -3), 
$h^2$*1e5 = 11.1079, 
$\mu$*1e5 = -5.554, 
$\eta$ = 3, 
$p_u$ = 0.3732, 
$p_m$ = 0.2491, 
$p_d$ = 0.3777, 
price=0.1726, 
$\Delta$ = 0.19247 ]  
0:  
[ index:(1, 0), 
$h^2$*1e5 = 10.5215, 
$\mu$*1e5 = -5.2608, 
$\eta$ = 3, 
$p_u$ = 0.3535, 
$p_m$ = 0.2888, 
$p_d$ = 0.3578, 
price=0.55274, 
$\Delta$ = 0.50304 ,  
index:(1, 0), 
$h^2$*1e5 = 10.5215, 
$\mu$*1e5 = -5.2608, 
$\eta$ = 3, 
$p_u$ = 0.3535, 
$p_m$ = 0.2888, 
$p_d$ = 0.3578, 
price=0.55274, 
$\Delta$ = 0.50304 ]  
3:  
[ index:(1, 3), 
$h^2$*1e5 = 11.1186, 
$\mu$*1e5 = -5.5593, 
$\eta$ = 3, 
$p_u$ = 0.3735, 
$p_m$ = 0.2484, 
$p_d$ = 0.3781, 
price=1.39859, 
$\Delta$ = 0.81218 ,  
index:(1, 3), 
$h^2$*1e5 = 11.1186, 
$\mu$*1e5 = -5.5593, 
$\eta$ = 3, 
$p_u$ = 0.3735, 
$p_m$ = 0.2484, 
$p_d$ = 0.3781, 
price=1.39859, 
$\Delta$ = 0.81218 ]  

2:  
-6:  
[ index:(2, -6), 
$h^2$*1e5 =  = 11.241, 
$\mu$*1e5 = -5.6205, 
$\eta$ = 3, 
$p_u$ = 0.3776, 
$p_m$ = 0.2401, 
$p_d$ = 0.3822, 
price=0.0, 
$\Delta$ = 0.0 ,  
index:(2, -6), 
$h^2$*1e5 = 11.241, 
$\mu$*1e5 = -5.6205, 
$\eta$ = 3, 
$p_u$ = 0.3776, 
$p_m$ = 0.2401, 
$p_d$ = 0.3822, 
price=0.0, 
$\Delta$ = 0.0 ]  
-3:  
[ index:(2, -3), 
$h^2$*1e5 = 10.6546, 
$\mu$*1e5 = -5.3273, 
$\eta$ = 3, 
$p_u$ = 0.3579, 
$p_m$ = 0.2798, 
$p_d$ = 0.3623, 
price=0.0, 
$\Delta$ = 0.0 ,  
index:(2, -3), 
$h^2$*1e5 = 10.7135, 
$\mu$*1e5 = -5.3567, 
$\eta$ = 3, 
$p_u$ = 0.3599, 
$p_m$ = 0.2758, 
$p_d$ = 0.3643, 
price=0.0, 
$\Delta$ = 0.0 ]  
0:  
[ index:(2, 0), 
$h^2$*1e5 = 10.1269, 
$\mu$*1e5 = -5.0634, 
$\eta$ = 3, 
$p_u$ = 0.3402, 
$p_m$ = 0.3154, 
$p_d$ = 0.3444, 
price=0.41631, 
$\Delta$ = 0.50304 ,  
index:(2, 0), 
$h^2$*1e5 = 11.2518, 
$\mu$*1e5 = -5.6259, 
$\eta$ = 3, 
$p_u$ = 0.378, 
$p_m$ = 0.2394, 
$p_d$ = 0.3826, 
price=0.46255, 
$\Delta$ = 0.50304 ]  
3:  
[ index:(2, 3), 
$h^2$*1e5 = 10.6642, 
$\mu$*1e5 = -5.3321, 
$\eta$ = 3, 
$p_u$ = 0.3582, 
$p_m$ = 0.2791, 
$p_d$ = 0.3626, 
price=1.22372, 
$\Delta$ = 1.0 ,  
index:(2, 3), 
$h^2$*1e5 = 10.7237, 
$\mu$*1e5 = -5.3619, 
$\eta$ = 3, 
$p_u$ = 0.3602, 
$p_m$ = 0.2751, 
$p_d$ = 0.3647, 
price=1.22372, 
$\Delta$ = 1.0 ]  
6:  
[ index:(2, 6), 
$h^2$*1e5 = 11.2614, 
$\mu$*1e5 = -5.6307, 
$\eta$ = 3, 
$p_u$ = 0.3783, 
$p_m$ = 0.2388, 
$p_d$ = 0.3829, 
price=2.46242, 
$\Delta$ = 1.0 ,  
index:(2, 6), 
$h^2$*1e5 = 11.2614, 
$\mu$*1e5 = -5.6307, 
$\eta$ = 3, 
$p_u$ = 0.3783, 
$p_m$ = 0.2388, 
$p_d$ = 0.3829, 
price=2.46242, 
$\Delta$ = 1.0 ]  

3:  
-9:[index:(3, -9), 
$h^2$*1e5 = 11.3607, 
$\mu$*1e5 = -5.6803, 
$\eta$ = 3, 
$p_u$ = 0.3816, 
$p_m$ = 0.232, 
$p_d$ = 0.3863, 
price=0, 
$\Delta$ = inf]  
-6:[index:(3, -6), 
$h^2$*1e5 = 10.7744, 
$\mu$*1e5 = -5.3872, 
$\eta$ = 3, 
$p_u$ = 0.3619, 
$p_m$ = 0.2717, 
$p_d$ = 0.3664, 
price=0, 
$\Delta$ = inf]  
-3:[index:(3, -3), 
$h^2$*1e5 = 10.2467, 
$\mu$*1e5 = -5.1233, 
$\eta$ = 3, 
$p_u$ = 0.3442, 
$p_m$ = 0.3073, 
$p_d$ = 0.3484, 
price=0, 
$\Delta$ = inf]  
0:[index:(3, 0), 
$h^2$*1e5 = 9.7717, 
$\mu$*1e5 = -4.8858, 
$\eta$ = 3, 
$p_u$ = 0.3283, 
$p_m$ = 0.3395, 
$p_d$ = 0.3323, 
price=0.0, 
$\Delta$ = inf]  
3:[index:(3, 3), 
$h^2$*1e5 = 10.2553, 
$\mu$*1e5 = -5.1277, 
$\eta$ = 3, 
$p_u$ = 0.3445, 
$p_m$ = 0.3068, 
$p_d$ = 0.3487, 
price=1.22372, 
$\Delta$ = inf]  
6:[index:(3, 6), 
$h^2$*1e5 = 10.7928, 
$\mu$*1e5 = -5.3964, 
$\eta$ = 3, 
$p_u$ = 0.3626, 
$p_m$ = 0.2704, 
$p_d$ = 0.367, 
price=2.46242, 
$\Delta$ = inf]  
9:[index:(3, 9), 
$h^2$*1e5 = 11.39, 
$\mu$*1e5 = -5.695, 
$\eta$ = 3, 
$p_u$ = 0.3826, 
$p_m$ = 0.2301, 
$p_d$ = 0.3873, 
price=3.71628, 
$\Delta$ = inf]  

A hedger is short 10,000 European calls.  
As an option covers 100 shares of stock, N = 1,000,000.  


0:
0:
[index:(0, 0), 
$h^2$*1e5 = 10.96, 
mu=-5.48, 

$\eta$ = 3, 
$p_u$ = 0.3682, 
$p_m$ = 0.2591, 
$p_d$ = 0.3727, 
price=0.56263, 
$\Delta$ = 0.50304]

In [1]:
N = 1000000
delta = 0.50304
stock_number = N*delta